In [1]:
! ls

 app.ipynb	    DenvtoMinnAGG.txt   SeatoDenvAGG.txt
 AtltoDalAGG.txt    LAtoDenvAGG.txt    'traffic engineering.ipynb'
 ChitoAtlAGG.txt    MiatoAtlAGG.txt
 DaltoDenvAGG.txt   NYtoAtlAGG.txt


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch_geometric
from torch.nn import Linear, Module, BatchNorm1d, ReLU, Dropout, MSELoss, ModuleList, LSTM, L1Loss
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch.optim import Adam

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
seed = 42
import random
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# Data

## Flow

In [5]:
import os 
# device = 'cpu'
flows = []
for file in os.listdir()[:-1]:
    if file.endswith('.txt'):
        flow = pd.read_csv(file).values[:, -1]
        print(file)
        print(flow.shape)
        flows.append(flow)

flows = torch.from_numpy(np.stack(flows)).float()
# flows = torch.log2(flows + 1)
flows /= 1e6

SeatoDenvAGG.txt
(150,)
DenvtoMinnAGG.txt
(150,)
ChitoAtlAGG.txt
(150,)
LAtoDenvAGG.txt
(150,)
AtltoDalAGG.txt
(150,)
DaltoDenvAGG.txt
(150,)
NYtoAtlAGG.txt
(150,)
MiatoAtlAGG.txt
(150,)


In [6]:
flows.shape

torch.Size([8, 150])

## Graph

In [7]:
# device = 'cpu'
edge_index = torch.tensor([[0, 1, 1, 2, 0, 2, 4, 2, 3, 2, 3, 4, 3, 5, 6, 6, 5, 4, 7, 4, 7, 6, 5, 8, 6, 8],
                           [1, 0, 2, 1, 2, 0, 2, 4, 2, 3, 4, 3, 5, 3, 5, 6, 4, 6, 4, 7, 6, 7, 8, 5, 8, 6]], dtype=torch.long)
x = torch.tensor([[1], [1], [1], [1], [1], [1], [1], [1], [1]], dtype=torch.float)

data = Data(x=x.to(device), edge_index=edge_index.to(device), flows=flows.to(device))


# Model

In [8]:
def train(st, data, time, loss_fn, optimizer):
    st.train()

    optimizer.zero_grad()

    output = st(data, time)

    loss = loss_fn(output, data.flows[:, time])

    loss.backward()

    optimizer.step()

    return loss.item()
    

In [9]:
@torch.no_grad()
def eval(st, data, times, ret_outputs=None):
    st.eval()

    errors = []
    outputs = []
    for time in times:
        output = st(data, time)
        if ret_outputs:
            outputs.append(output)
        error = L1Loss()(output, data.flows[:, time])
        errors.append(error)
        # print(output, data.flows[:, time])
    if ret_outputs:
        return outputs
    return torch.tensor(errors).mean()

In [10]:
class SpatioTemporal(Module):
    def __init__(self, input_dim, gnn_dims, node_pooling_dims, graph_pooling_dims, rnn_dim, ff_dims, gnn_dropout_rate, ff_dropout_rate, rnn_dropout_rate):
        super(SpatioTemporal, self).__init__()
        self.input_dim = input_dim
        self.gnn_dims = gnn_dims
        self.node_pooling_dims = node_pooling_dims
        self.graph_pooling_dims = graph_pooling_dims
        self.rnn_dim = rnn_dim
        self.ff_dims = ff_dims
        self.gnn_dropout_rate = gnn_dropout_rate
        self.ff_dropout_rate = ff_dropout_rate
        self.rnn_dropout_rate = rnn_dropout_rate

        ## GNN
        dims = [self.input_dim] + gnn_dims
        self.gnn_layers = ModuleList(
            [
               GCNConv(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        self.gnn_bn_layers = ModuleList(
            [
               BatchNorm1d(dim) for dim in dims[1:]
            ]
        )

        ## Node Pooling
        dims = [self.gnn_dims[-1]] + self.node_pooling_dims
        self.node_pooling_layers = ModuleList(
            [
               Linear(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        self.node_pooling_bn_layers = ModuleList(
            [
               BatchNorm1d(dim) for dim in dims[1:]
            ]
        )

        ## Graph Pooling
        dims = [self.node_pooling_dims[-1]] + self.graph_pooling_dims
        self.graph_pooling_layers = ModuleList(
            [
               Linear(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        ## RNN
        self.rnn = LSTM(self.graph_pooling_dims[-1] + self.ff_dims[-1], self.rnn_dim, 2, dropout=self.rnn_dropout_rate)


        dims = [self.rnn_dim] + self.ff_dims
        self.ff_layers = ModuleList(
            [
               Linear(dim, dims[i+1]) for i, dim in enumerate(dims[:-1])
            ]
        )

        self.activation = ReLU()
        self.gnn_dropout = Dropout(self.gnn_dropout_rate)
        self.ff_dropout = Dropout(self.ff_dropout_rate)

    def forward(self, data, timestep):
        x = data.x

        # GNN
        for conv, bn in zip(self.gnn_layers, self.gnn_bn_layers):
            
            x = conv(x, edge_index=data.edge_index)
            x = bn(x)
            x = self.activation(x)
            x = self.gnn_dropout(x)
        
        # Node Pooling 
        for ff, bn in zip(self.node_pooling_layers, self.node_pooling_bn_layers):
            x = ff(x)
            x = bn(x)
            x = self.activation(x)
            x = self.ff_dropout(x) 
        
        x = x.sum(0).reshape(1, -1)

        # Graph Pooling 
        for ff in self.graph_pooling_layers:
            x = ff(x)
            # x = bn(x)
            x = self.activation(x)
            x = self.ff_dropout(x) 

        # RNN
        x = x.repeat(timestep + 1, 1)

        global device
        x_time = torch.hstack((torch.zeros(flows.shape[0]).reshape(-1, 1).to(device), data.flows[:, :timestep])).t()

        x = torch.hstack((x_time, x))
        
        x, (h, c) = self.rnn(x)
        
        # Regression
        x = x[-1, :]
        for ff in self.ff_layers[:-1]:
            x = ff(x)
            # x = bn(x)
            x = self.activation(x)
            x = self.ff_dropout(x)
        
        x = self.ff_layers[-1](x)
        
        return self.ff_dropout(x).reshape(-1)



# Training

In [11]:
def pipeline_for_one_config(st, data, loss_fn, optimizer):
    train_errors = []
    val_errors = []
    test_errors = []
    train_errors.append(eval(st, data, range(1, 80)))
    val_errors.append(eval(st, data, range(81, 105)))
    test_errors.append(eval(st, data, range(106, 130)))

    test_outputs = []
    for epoch in range(100):
        train_errors.append(eval(st, data, range(1, 80)))
        val_errors.append(eval(st, data, range(81, 105)))
        test_errors.append(eval(st, data, range(106, 130)))
        test_outputs.append(eval(st, data, range(106, 130), True))

        epoch_loss = 0
        for time in range(1, 80):
            epoch_loss += train(st, data, time, loss_fn, optimizer)
        # if epoch % 5 == 0:
        #     print(f'Epoch {epoch + 1}') 
        #     print(f'Train Loss: {epoch_loss / 80 :.3f}')
        #     print(f'Train Error: {eval(st, data, range(1, 80)).item(): .2f}')
        #     print(f'Validation Error: {eval(st, data, range(81, 105)).item(): .2f}')
        #     print(f'Test Error: {eval(st, data, range(106, 130)).item() :.2f}')   
        #     print('===============================')
    train_errors.append(eval(st, data, range(1, 80)))
    val_errors.append(eval(st, data, range(81, 105)))
    test_errors.append(eval(st, data, range(106, 130)))

    i = torch.tensor(val_errors).argmin()
    return val_errors[i], test_errors[i], i, test_outputs[i]

In [12]:
gnn_dim = 32
np_dim = 160
gp_dim = 80
rnn_dim = 384
ff_dim = 300
dropout = 0.3
lr = 1e-4
st = SpatioTemporal(1, [gnn_dim, gnn_dim], [np_dim, np_dim], [gp_dim, gp_dim], rnn_dim, [ff_dim, 8], dropout, dropout, dropout).to(device)
optimizer = Adam(st.parameters(), lr=lr)
loss_fn = MSELoss()
dev_error, test_error, i, predictions = pipeline_for_one_config(st, data, loss_fn, optimizer)

In [13]:
first = torch.stack(predictions)[0, :]
keys = [(0, 2), (2, 3), (5, 6), (1, 2), (4, 6), (2, 4), (6, 8), (6, 7)]
graph = np.zeros(81).reshape(-1, 9)
for i in range(9):
    for j in range(9):
        for key, prediction in zip(keys, first):
            if i in key and j in key and i != j:
                graph[i][j] = prediction
                break
graph
####################################################
graph_before = np.zeros(81).reshape(-1, 9)
for i in range(9):
    for j in range(9):
        for edge in data.edge_index.T:
            if (i in edge) and (j in edge) and (i != j):
                graph_before[i][j] = 1
                
graph_before

array([[0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 1., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0.]])

In [14]:
from scipy.sparse.csgraph import dijkstra
te_dist, te_pre = dijkstra(graph, directed=False, min_only=False, return_predecessors=True)
dist, pre = dijkstra(graph_before, directed=False, min_only=False, return_predecessors=True)

cities = ['Seattle', 'LA', 'Denver', 'Minneapolis', 'Dallas', 'Chicago', 'Atlanta', 'Miami', 'NY']
te_pre = te_pre.astype(np.int32)
pre = pre.astype(np.int32)


In [15]:
def print_shortest_paths(predecessors, source):
    paths = []
    for vertex in range(len(predecessors)):
        if vertex != source:
            paths.append(print_shortest_path(predecessors, source, vertex))
    return paths

def print_shortest_path(predecessors, source, destination):
    path = []
    current_vertex = destination

    while current_vertex != -9999:
        path.insert(0, current_vertex)
        current_vertex = predecessors[current_vertex]

    print(f"Shortest path from {source} to {destination}: {path}")
    return path

# Example usage
te_paths = []
paths = []
for i in range(9):
    predecessors_matrix = te_pre[i, :].tolist()
    source_vertex = i

    te_path = print_shortest_paths(predecessors_matrix, source_vertex)
    te_paths.append(te_path)
    print('------------------------------------------------------')

    predecessors_matrix = pre[i, :].tolist()
    source_vertex = i
    path = print_shortest_paths(predecessors_matrix, source_vertex)
    paths.append(path)
    
    print('===============================================')



Shortest path from 0 to 1: [0, 2, 1]
Shortest path from 0 to 2: [0, 2]
Shortest path from 0 to 3: [0, 2, 3]
Shortest path from 0 to 4: [0, 2, 4]
Shortest path from 0 to 5: [0, 2, 4, 6, 5]
Shortest path from 0 to 6: [0, 2, 4, 6]
Shortest path from 0 to 7: [0, 2, 4, 6, 7]
Shortest path from 0 to 8: [0, 2, 4, 6, 8]
------------------------------------------------------
Shortest path from 0 to 1: [0, 1]
Shortest path from 0 to 2: [0, 2]
Shortest path from 0 to 3: [0, 2, 3]
Shortest path from 0 to 4: [0, 2, 4]
Shortest path from 0 to 5: [0, 2, 3, 5]
Shortest path from 0 to 6: [0, 2, 4, 6]
Shortest path from 0 to 7: [0, 2, 4, 7]
Shortest path from 0 to 8: [0, 2, 3, 5, 8]
Shortest path from 1 to 0: [1, 2, 0]
Shortest path from 1 to 2: [1, 2]
Shortest path from 1 to 3: [1, 2, 3]
Shortest path from 1 to 4: [1, 2, 4]
Shortest path from 1 to 5: [1, 2, 4, 6, 5]
Shortest path from 1 to 6: [1, 2, 4, 6]
Shortest path from 1 to 7: [1, 2, 4, 6, 7]
Shortest path from 1 to 8: [1, 2, 4, 6, 8]
------------

In [18]:
first = np.zeros((9, 9))
for path, te_path in zip(paths, te_paths):
    if path != te_path:
        for i in range(8):
            if path[i] != te_path[i] and not first[path[i][0]][path[i][-1]]:
                first[path[i][0]][path[i][-1]] = 1
                first[path[i][-1]][path[i][0]] = 1
                print(f'Path from {cities[path[i][0]]} to {cities[path[i][-1]]}')
                print('  Without Traffic Engineering: ')
                for k in path[i]:
                    print(f'{cities[k]}', end=', ')
                print('\n  With Traffic Engineering: ')
                for k in te_path[i]:
                    print(f'{cities[k]}', end=', ')
                print('\n===================')

Path from Seattle to LA
  Without Traffic Engineering: 
Seattle, LA, 
  With Traffic Engineering: 
Seattle, Denver, LA, 
Path from Seattle to Chicago
  Without Traffic Engineering: 
Seattle, Denver, Minneapolis, Chicago, 
  With Traffic Engineering: 
Seattle, Denver, Dallas, Atlanta, Chicago, 
Path from Seattle to Miami
  Without Traffic Engineering: 
Seattle, Denver, Dallas, Miami, 
  With Traffic Engineering: 
Seattle, Denver, Dallas, Atlanta, Miami, 
Path from Seattle to NY
  Without Traffic Engineering: 
Seattle, Denver, Minneapolis, Chicago, NY, 
  With Traffic Engineering: 
Seattle, Denver, Dallas, Atlanta, NY, 
Path from LA to Chicago
  Without Traffic Engineering: 
LA, Denver, Minneapolis, Chicago, 
  With Traffic Engineering: 
LA, Denver, Dallas, Atlanta, Chicago, 
Path from LA to Miami
  Without Traffic Engineering: 
LA, Denver, Dallas, Miami, 
  With Traffic Engineering: 
LA, Denver, Dallas, Atlanta, Miami, 
Path from LA to NY
  Without Traffic Engineering: 
LA, Denver, Minn